# Data Prep

In [ ]:
# any Pip installs we need for this project
!pip install sklearn
!pip install pandas
!pip install earthpy


In [ ]:
# Libraries we need for this project
import numpy as np
import pandas as pd
import os
import earthpy as et
import math
import statistics
from statistics import stdev

# from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [6]:
# data prep functions

def get_arrest_data():
    # I just picked the biggest dataset - this will take a few minutes to download. luckly you will only need to 
    # download this once
    file_url = "https://stacks.stanford.edu/file/druid:yg821jf8611/yg821jf8611_ca_statewide_2020_04_01.csv.zip"
    data_file = et.data.get_data(url=file_url)
    fname = os.path.join(data_file, "ca_statewide_2020_04_01.csv")
    return pd.read_csv(fname, on_bad_lines='skip')

def shuffle_data(data, seed):
    np.random.seed(seed)
    try:
        np.random.shuffle(data.values)
    except:
        np.random.shuffle(data)
    return(data)

# splits data into training(2/3) and validation(1/3) dataframes
def split_data(data_frame):
    training_length = round(2/3 * len(data_frame))

    training_df = data_frame[:training_length]
    validation_df = data_frame[training_length:]

    return training_df, validation_df

# assuming that that yhat column is the last column in the dataset, this returns the features seperated from
# the yhat column. for example to use this function: 'arrest_feat_df, arrest_yhat_df = split_data_yhat(df)'
def split_data_yhat(df):
    # This function will not work on our dataset - our yhat is 'arrest_made' column and it is not the 
    # last column of the dataset. so if someone needs this - you will have to re-write this function.
    return df.iloc[:,:-1], df.iloc[:,-1]


    

In [ ]:
# Use this for testing
arrest_data_df = Data_prep.get_arrest_data()
print(arrest_data_df)